<a href="https://colab.research.google.com/github/rawanmt/ECC/blob/main/ECC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii

def encrypt_AES_GCM(msg, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (ciphertext, aesCipher.nonce, authTag)

def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

# curve = registry.get_curve('brainpoolP256r1')

field = SubGroup(p=37, g=(2, 5), n=18, h=1)

curve = Curve(a=7, b=3, field=field, name='p1707')

def encrypt_ECC(msg, pubKey):
    ciphertextPrivKey = secrets.randbelow(curve.field.n)
    sharedECCKey = ciphertextPrivKey * pubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
    ciphertextPubKey = ciphertextPrivKey * curve.g
    return (ciphertext, nonce, authTag, ciphertextPubKey)

def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext, nonce, authTag, ciphertextPubKey) = encryptedMsg
    sharedECCKey = privKey * ciphertextPubKey
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
    return plaintext

msg = b'Hello World' \

print("Original Messsage:", msg, "\n")
privKey = secrets.randbelow(curve.field.n)
pubKey = privKey * curve.g

encryptedMsg = encrypt_ECC(msg, pubKey)

encryptedMsgObj = {
    'Cipher Text': binascii.hexlify(encryptedMsg[0]),
    'Nonce': binascii.hexlify(encryptedMsg[1]),
    'Digital Signiture': binascii.hexlify(encryptedMsg[2]),
    'Ciphertext Public Key': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
}

for key, value in encryptedMsgObj.items() :
    print(key, value, "\n")

decryptedMsg = decrypt_ECC(encryptedMsg, privKey)
print("Decrypted Messsage:", decryptedMsg, "\n")




Original Messsage: b'Hello World' 

Cipher Text b'ddc32ec3bca8f015f30c77' 

Nonce b'62310ae854f0b7ad6a7a67caeda0a5b0' 

Digital Signiture b'88c65bc8ae1288ff1569ce38e5b76560' 

Ciphertext Public Key 0x171 

Decrypted Messsage: b'Hello World' 

